In [ ]:
import os

import openai
import numpy as np
from dotenv import load_dotenv
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct

In [ ]:
load_dotenv('.env')

In [ ]:
openai.api_key = os.getenv('OPENAI_API_KEY')

In [ ]:
client = QdrantClient(host="localhost", port=6333)

In [ ]:
client.recreate_collection(
    collection_name="docs",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

In [ ]:
docs_info = client.get_collection("docs")
print(docs_info.dict())

In [ ]:
def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding'] # type: ignore

In [ ]:
texts = [
    "中村弘武は32歳の東京都在住です。エムスリーという企業で働いでいます。趣味は麻雀、サウナ、筋トレです。",
    "大垣慶介は機械学習分野の研究開発を10年間続け、研究者として国際学会に論文を投稿する傍ら、エンジニアとしてAIを主軸においたゲームの技術責任者やwebサービスの新機能開発を行ってきた。エムスリーでは入社後3ヶ月で行動ログ解析の新システムを立ち上げ、製薬企業のマーケティング効率化に貢献、経営陣からのバックアップもあり、専用チームをリードしている。同時に研究の専門性を活かし、臨床AIの企画・開発を行っている。",
    "北川遼はエムスリー2021年新卒入社。学生時代はヒューマンロボットインタラクションの分野を研究し、その分野のトップカンファレンスに採択されたOSSにも精力的に貢献し、大学院時代に10個以上の貢献を行った大学院時代のアルバイトではデータ基盤を作成し、アルバイトながら社内で表彰された",
    "池島は前職ではコンサルティング会社において、クライアント企業のDXを目的とした機械学習PoCプロジェクトをリード。戦略の立案から機械学習モデルの開発まで深く関与。エムスリーでは医師の嗜好・性格予測するAIシステムの開発に従事。クライアント企業のマーケティング効率化やエムスリーの自社サービスの体験向上に活用中。"
]

In [ ]:
embeddings = [get_embedding(t) for t in texts]

In [ ]:
client.upsert(
    collection_name="docs",
    points=[
        PointStruct(
            id=idx,
            vector=vector,
        )
        for idx, vector in enumerate(embeddings)
    ]
)